In [5]:
%pip install --user gym
%pip install pygame --pre
%pip install gym[box2d]
%pip install numpy
%pip install -U matplotlib
%pip install pynput
%pip install keyboard


import gym
from gym import wrappers
import keyboard
import datetime as dt
from datetime import datetime
from random import random
    
main_loop_time = 0.02 # measured in seconds, 50 fps

keys_being_pressed = {"w": False, "a": False, "s": False, "d": False}

def choosePlayerActionDiscrete(): # choose ONE action based on keys_being_pressed 
    
    if keys_being_pressed['s']: # brake pressed
        return 4 # brake
    elif keys_being_pressed['w']: # brake off, gas pressed
        return 3 # gas
    else: #just steering 
        if (keys_being_pressed['a'] and keys_being_pressed['d']) or not(keys_being_pressed['a'] or keys_being_pressed['d']): # both or neither
            return 0 # do nothing, steering cancels out
        elif keys_being_pressed['a']: # turn left ... these are backwards???
            return 2
        else: # turn right
            return 1

def nondeterminism(action, chance_of_mistakes=0.05):
    roll = random()
    if roll - chance_of_mistakes <= 0: # chance to steer wrong way if you're steering
        if action == 2:
            return 1
        if action == 1:
            return 2
        
    elif roll - (2 * chance_of_mistakes) <= 0: # chance for temporary brake failure 
        if action == 4:
            return 0
        
    return action
    
    

#target env
env = gym.make('CarRacing-v2', render_mode='human', continuous=False, domain_randomize=False)

# If continuous = False, actions are:
# 0: do nothing
# 1: steer left   (pretty sure this is actually steer right and vice versa)
# 2: steer right
# 3: gas
# 4: brake

# If continuous = True, actions are:
# of type numpy.ndarray[] (length 3) 
# three floats: 
# index 0: steering, -1 is full left, +1 is full right
# index 1: gas
# index 2: braking

#env.reset(seed=3)
env.reset()

end_loop = False
max_iterations = 1000000 # just in case
i = 0

while (i < max_iterations and not end_loop): #main gameplay loop
    i += 1
    
    for key in ['w', 'a', 's', 'd']:
        keys_being_pressed[key] = keyboard.is_pressed(key)

    if (keyboard.is_pressed('q')):
        end_loop = True

    start_time = datetime.now()
    target_end_time = start_time + dt.timedelta(seconds=main_loop_time)
    
    while datetime.now() < target_end_time: # wait for loop time
        pass

    #show image
    env.render()
    
    action = choosePlayerActionDiscrete()
    
    #normal levels of nondeterminism
    action = nondeterminism(action, chance_of_mistakes=0.05)   
    
    # comment out the line above, uncomment this, and run for very nondeterministic fun 
    #action = nondeterminism(action, chance_of_mistakes=0.2)
    
    env.step(action)
env.close()



Note: you may need to restart the kernel to use updated packages.


ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
